In [31]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import json
import jax.numpy as np
import numpy as onp
import jax
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from timecast.learners import AR
from timecast.learners._ar import _ar_predict, _ar_batch_window
from timecast.utils.numpy import ecdf
from timecast.utils.losses import MeanSquareError
import torch
import matplotlib

plt.rcParams['figure.figsize'] = [20, 10]

import tqdm.notebook as tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
from ealstm.gaip import FloodLSTM
from ealstm.gaip import FloodData
from ealstm.gaip.utils import MSE, NSE

from timecast.optim import SGD
from timecast.learners import AR

In [3]:
cfg_path = "/home/dsuo/src/toy_flood/ealstm/runs/run_2503_0429_seed283956/cfg.json"

In [4]:
flood_lstm = FloodLSTM(cfg_path)

In [13]:
flood_data = FloodData(cfg_path)
ea_data = pickle.load(open("../ealstm/runs/run_2503_0429_seed283956/lstm_seed283956.p", "rb"))

In [10]:
results = {}
date_range = pd.date_range(start=flood_data.cfg["val_start"], end=flood_data.cfg["val_end"])
for X, y, basin in tqdm.tqdm(flood_data.generator(), total=len(flood_data.basins)):
    pred = flood_lstm.predict(X)
    true = y[flood_data.cfg["seq_length"] - 1:]
    df = pd.DataFrame(data={"qobs": true.ravel(), "qsim": pred.ravel()}, index=date_range)
    results[basin] = df

In [18]:
for basin in tqdm.tqdm(flood_data.basins):
    onp.testing.assert_array_almost_equal(results[basin], ea_data[basin], decimal=4)

In [145]:
from ealstm.gaip import FloodLSTM
from ealstm.gaip import FloodData
from ealstm.gaip.utils import MSE, NSE

from timecast.optim import SGD
from timecast.learners import AR

cfg_path = "/home/dsuo/src/toy_flood/ealstm/runs/run_2503_0429_seed283956/cfg.json"
flood_data = FloodData(cfg_path)

LR_AR = 1e-5
AR_INPUT_DIM=32
AR_OUTPUT_DIM=1

results = {}
mses = []
nses = []
for X, _, basin in tqdm.tqdm(flood_data.generator(), total=len(flood_data.basins)):
    sgd = SGD(learning_rate=LR_AR, online=False)
    ar = AR(input_dim=AR_INPUT_DIM,
            output_dim=AR_OUTPUT_DIM,
            window_size=flood_data.cfg["seq_length"],
            optimizer=sgd,
            history=X[:flood_data.cfg["seq_length"]],
            fit_intercept=True,
            constrain=False
           )
    # NOTE: difference in indexing convention, so need to pad one row
    X = np.vstack((X[flood_data.cfg["seq_length"]:], np.ones((1, X.shape[1]))))
    Y = np.array(ea_data[basin].qobs).reshape(-1, 1)
    
    Y_lstm = np.array(ea_data[basin].qsim).reshape(-1, 1)
    Y_target = Y - Y_lstm
    
    Y_ar = ar.predict_and_update(X, Y_target)
    
    Y_hat = Y_lstm + Y_ar
    
    mse = MSE(Y, Y_hat)
    nse = NSE(Y, Y_hat)
    results[basin] = {
        "mse": mse,
        "nse": nse,
        "count": X.shape[0],
        "avg_mse": np.mean(np.array(mses)),
        "avg_nse": np.mean(np.array(nses))
    }
    mses.append(mse)
    nses.append(nse)
    print(basin, mse, nse, np.mean(np.array(mses)), np.mean(np.array(nses)))

01022500 0.59074104 0.8800156 0.59074104 0.8800156
01031500 1.0027689 0.8932544 0.79675496 0.886635
01047000 1.4249043 0.8664847 1.0061381 0.8799183
01052500 1.7676423 0.8650273 1.1965141 0.87619555
01054200 7.349815 0.7506206 2.4271743 0.85108054
01055000 4.5261536 0.7442876 2.7770042 0.8332817
01057000 0.9297007 0.8734561 2.5131037 0.83902085
01073000 0.88541025 0.86551344 2.309642 0.8423324
01078000 0.6426703 0.89295614 2.124423 0.8479573
01123000 0.79172385 0.83917654 1.991153 0.84707916
01134500 1.8118598 0.77957004 1.9748535 0.84094197
01137500 2.5516274 0.7490745 2.022918 0.83328635
01139000 0.7919953 0.7776362 1.9282317 0.8290056
01139800 0.87930524 0.7588776 1.8533084 0.8239965
01142500 1.0633992 0.78462267 1.8006477 0.8213716
01144000 0.6678886 0.8641338 1.7298503 0.8240442
01162500 0.8214795 0.8384537 1.6764168 0.8248918
01169000 2.5450714 0.7171354 1.7246753 0.81890535
01170100 2.061385 0.74497557 1.742397 0.8150143
01181000 2.0647883 0.7748269 1.7585166 0.81300485
01187300

02479155 6.1671915 0.6715582 2.0246162 0.72822964
02479300 1.7246664 0.79846334 2.0228307 0.7286477
02479560 1.2151204 0.8650907 2.0180511 0.72945505
02481000 6.9328938 0.72077584 2.046962 0.729404
02481510 2.3501818 0.80902 2.0487356 0.72986966
04015330 1.6093724 0.7583636 2.046181 0.7300353
04024430 0.56316054 0.75025517 2.0376086 0.7301522
04027000 0.23686928 0.9015581 2.0272596 0.7311372
04040500 0.32109022 0.8522735 2.01751 0.73182946
04043050 1.2561044 0.71657926 2.0131838 0.73174286
04045500 0.023713898 0.9726626 2.0019438 0.7331039
04057510 0.031701587 0.9581821 1.9908752 0.7343684
04057800 0.46327338 0.7893436 1.982341 0.7346756
04059500 0.1059656 0.9034806 1.9719167 0.73561335
04063700 0.0994144 0.85112756 1.9615715 0.7362516
04074950 0.03758232 0.8577359 1.951 0.7369191
04105700 0.023834297 0.85692596 1.940469 0.7375748
04115265 0.09350471 0.66496 1.9304312 0.73718023
04122200 0.025303455 0.87288666 1.920133 0.7379137
04122500 0.015869053 0.91026664 1.9098951 0.7388404
04127

06910800 1.3998089 0.8601829 1.9753324 0.72673076
06911900 1.8032906 0.8462194 1.9748156 0.72708964
06917000 3.0592127 0.73808503 1.9780623 0.72712255
06918460 1.0149873 0.83525664 1.9751877 0.7274453
06919500 1.0639044 0.8697022 1.9724755 0.7278687
06921070 1.7818078 0.83782023 1.9719096 0.72819495
06921200 5.6025004 0.6542084 1.982651 0.7279761
07057500 0.8621073 0.7056042 1.9793458 0.72791004
07060710 1.8745272 0.5943667 1.9790375 0.7275173
07066000 1.9107823 0.6101496 1.9788373 0.72717315
07083000 0.12757829 0.97217035 1.9734242 0.7278895
07142300 0.017392686 0.25925893 1.9677215 0.7265233
07145700 1.4384717 0.73391557 1.966183 0.72654474
07167500 10.846441 0.38681984 1.9919229 0.72556
07180500 2.133334 0.68597066 1.9923314 0.7254456
07184000 9.047254 0.70604694 2.0126624 0.72538966
07195800 2.6854558 0.5764644 2.014596 0.72496176
07196900 9.49298 0.5034734 2.0360239 0.7243271
07197000 1.3103372 0.844996 2.0339506 0.7246719
07208500 0.031855457 0.8181125 2.0282466 0.7249381
0726100

14305500 16.051962 0.862204 2.6510873 0.7158072
14306340 8.902316 0.86327004 2.66364 0.7161033
14306500 3.6019247 0.9106377 2.6655202 0.7164932
14308990 1.4801569 0.8436923 2.6631494 0.7167476
14309500 3.9852526 0.85496086 2.6657884 0.71702343
14316700 4.500821 0.8372748 2.6694438 0.717263
14325000 7.90914 0.8693719 2.6798606 0.7175655
14362250 0.13616656 0.88214505 2.6748137 0.717892
14400000 16.560295 0.9097344 2.7023096 0.7182719
10259000 0.38615602 0.41322958 2.6977322 0.71766907
11124500 2.123736 0.17217112 2.6966002 0.7165931
11141280 0.64602184 0.5993042 2.6925633 0.7163622
11143000 3.3452678 0.87368906 2.693846 0.7166713
11148900 6.6102858 0.74480796 2.701525 0.7167264
11151300 0.3225834 0.095182 2.6968696 0.7155101
11176400 0.93579197 0.6161082 2.69343 0.71531594
11230500 0.3179561 0.9616319 2.6887996 0.7157961
11237500 2.1952934 0.8995956 2.6878393 0.7161537
11264500 0.8062822 0.92797846 2.6841857 0.7165651
11266500 0.94838077 0.91722393 2.680822 0.7169539
11284400 2.4877527 